In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!ls "/content/drive/My Drive"


In [ ]:
# pip install transformers datasets

In [ ]:
import os
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
import torch  # Necessário para verificar o dispositivo (GPU/CPU)

# Configuração do modelo e tokenizer
model_name = "gpt2"  # Nome do modelo pré-treinado (GPT-2 neste caso).
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Verificar se há GPU disponível e mover o modelo para o dispositivo correto
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Adicionar um token de padding ao tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Caminho para o dataset preparado
dataset_path = "/content/drive/My Drive/trn_filtered.json"  # Certifique-se de que este arquivo foi gerado na etapa anterior.
dataset = load_dataset("json", data_files=dataset_path)

# Limitar o dataset para 100.000 exemplos
max_samples = 100_000
dataset["train"] = dataset["train"].select(range(min(len(dataset["train"]), max_samples)))

# Função para tokenizar os dados
def preprocess_function(examples):
    """
    Concatena os campos de entrada ('input') e saída ('output') com o token EOS (<|endoftext|>)
    e realiza a tokenização necessária.
    """
    inputs = [inp + tokenizer.eos_token + out for inp, out in zip(examples["input"], examples["output"])]
    return tokenizer(inputs, truncation=True, padding="max_length", max_length=128)

# Aplicar a tokenização ao dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

# Configurar o Data Collator
# Responsável por criar lotes de dados para entrada no modelo.
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # mlm=False porque GPT-2 é um modelo causal, não usa aprendizado de máscara.
)

# Configurações do treinamento
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",  # Diretório para salvar os resultados do treinamento.
    overwrite_output_dir=True,  # Permitir sobrescrever saídas anteriores.
    learning_rate=5e-5,  # Taxa de aprendizado para o otimizador.
    per_device_train_batch_size=16,  # Aumentar o tamanho do lote para aproveitar a GPU.
    gradient_accumulation_steps=4,  # Ajustado para balancear memória e tamanho efetivo do lote.
    num_train_epochs=2,  # Número de épocas reduzido.
    save_strategy="no",  # Não salvar checkpoints durante o treinamento.
    logging_dir="./logs",  # Diretório para salvar logs.
    logging_steps=500,  # Frequência de log durante o treinamento.
    report_to="none",  # Evitar problemas com integrações externas como WandB.
    fp16=True,  # Ativar Mixed Precision para acelerar o treinamento em GPUs com suporte.
)

# Configurar o Trainer
trainer = Trainer(
    model=model,  # Modelo que será ajustado.
    args=training_args,  # Argumentos configurados para o treinamento.
    train_dataset=tokenized_dataset["train"],  # Dados de treinamento.
    tokenizer=tokenizer,  # Tokenizer para processar os dados.
    data_collator=data_collator,  # Collator para criar lotes de treinamento.
)

# Executar o treinamento
trainer.train()

# Salvar o modelo ajustado
model.save_pretrained("./gpt2-finetuned")  # Salvar os pesos do modelo ajustado.
tokenizer.save_pretrained("./gpt2-finetuned")  # Salvar o tokenizer ajustado.

print("Fine-tuning concluído! O modelo foi salvo no diretório './gpt2-finetuned'.")


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

<ipython-input-9-f3d34fd60f2e>:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,3.398200
1000,3.250100
1500,3.208400
2000,3.137800
2500,3.121000
3000,3.113200


In [ ]:
!zip -r gpt2-finetuned.zip ./gpt2-finetuned

  adding: gpt2-finetuned/ (stored 0%)
  adding: gpt2-finetuned/vocab.json (deflated 59%)
  adding: gpt2-finetuned/generation_config.json (deflated 24%)
  adding: gpt2-finetuned/config.json (deflated 51%)
  adding: gpt2-finetuned/model.safetensors (deflated 7%)
  adding: gpt2-finetuned/tokenizer.json (deflated 82%)
  adding: gpt2-finetuned/tokenizer_config.json (deflated 54%)
  adding: gpt2-finetuned/special_tokens_map.json (deflated 60%)
  adding: gpt2-finetuned/merges.txt (deflated 53%)
